In [2]:
import os
import sys

import numpy as np 
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [84]:
# demo_icp_clouds = o3d.data.DemoICPPointClouds()
src = o3d.io.read_point_cloud("src.ply")
dst = o3d.io.read_point_cloud("dst.ply")
P = np.asarray(src.points)
Q = np.asarray(dst.points)

print(P)
print(Q)
print(P.shape)
print(Q.shape)

[[1.01953125 0.88671875 2.27726722]
 [1.04296875 0.88671875 2.27699804]
 [1.05859375 0.88671875 2.27722216]
 ...
 [1.26609075 2.37890625 2.26171875]
 [1.26171875 2.38015151 2.26171875]
 [1.26171875 2.37890625 2.26367211]]
[[0.27405298 1.02095513 1.12868408]
 [0.29389196 1.01755603 1.1406841 ]
 [0.30689903 1.01520229 1.14901173]
 ...
 [0.49888183 2.42827751 1.63517043]
 [0.49521603 2.43012446 1.63321714]
 [0.49414874 2.42849813 1.63447699]]
(198835, 3)
(198835, 3)


In [88]:
# Kabsch algorithm for unique random pairs

print(P.shape[0])
N_all = P.shape[0] - P.shape[0]%P.shape[1]
print(N_all) # 0 ~ 198833
stride = int(N_all/3)
print(stride)

# unique random 3 indices
SEED = 0 # deterministic
indices_random = np.random.default_rng(seed=SEED).choice(P.shape[0], N_all, replace=False).reshape((stride, P.shape[1]))
print(indices_random)
print(indices_random.shape)

# select from indices
P_selected = np.asmatrix(P[indices_random[0]])
# P_selected = np.asmatrix(P[0])
print(P_selected)
print(type(P_selected))
Q_selected = np.asmatrix(Q[indices_random[0]])
# Q_selected = np.asmatrix(Q[0])
print(Q_selected)
print(type(Q_selected))

print(P_selected.shape)
print(Q_selected.shape)

198835
198834
66278
[[ 78262  67929 134603]
 [160826  67523 189929]
 [158760 124440 183588]
 ...
 [ 34847   3286  14959]
 [  8146  61206  53642]
 [101630 126649 169133]]
(66278, 3)
[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
<class 'numpy.matrix'>
[[1.46072831 1.74892238 1.60820396]
 [1.47696625 1.63610562 1.5760181 ]
 [2.34348337 2.35300524 1.38992812]]
<class 'numpy.matrix'>
(3, 3)
(3, 3)


In [89]:
def rigid_transform_3D(A, B, scale):

    assert len(A) == len(B)
    N = A.shape[0]

    # mean of each point cloud
    Am = np.mean(A, axis=0)
    Bm = np.mean(B, axis=0)

    # if len(Am.shape) == 1: Am = Am.reshape(1, N)
    # if len(Bm.shape) == 1: Bm = Bm.reshape(1, N)

    # centered point clouds
    Ac = A - np.tile(Am, (N, 1))
    Bc = B - np.tile(Bm, (N, 1))

    H = np.transpose(Bc) * Ac # NOTE: Kabsch; H = P^T \cdot Q
    if scale: H /= N

    """
    Based on rotation formula, optimal rotation R is

    R = sqrt(H^T \cdot H) \cdot H^-1

    since directly solving this is complicated and hard to handle edge cases (e.g., singular H), use SVD
    """

    U, S, Vt = np.linalg.svd(H) # NOTE: Kabsch; H = USV^T
    
    R = Vt.T * U.T # NOTE: Kabsch; R = V \cdot U^T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("[DEBUG] Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    
    if scale:
        Avar = np.var(A, axis=0).sum()
        c = 1 / (1 / Avar * np.sum(S)) # scale singular value
        t = -R.dot(Bm.T * c) + Am.T
        
    else:
        c = 1
        t = -R.dot(Bm.T) + Am.T

    return c, R, t


s, R, t = rigid_transform_3D(P_selected, Q_selected, False)
print(R)
print(t)

[DEBUG] Reflection detected
[[ 0.84037556 -0.1469134   0.52171388]
 [ 0.00625204  0.96512718  0.26170677]
 [-0.54196848 -0.21667019  0.81198781]]
[[ 0.35045859]
 [-0.39533494]
 [ 1.73079657]]


In [90]:
P_selected2 = (R * Q_selected.T) + np.tile(t, (1, 3))
P_selected2 = P_selected2.T
print(P_selected)
print(P_selected2)

[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.16010116 1.72260799 1.86603053]
 [2.17352962 1.60540373 1.85553958]
 [2.69932154 2.25401953 1.07948104]]


In [91]:
# export point clouds

np.savetxt("P_selected.txt", P_selected)
np.savetxt("Q_selected.txt", Q_selected)

In [93]:
# TODO: check ordering

T = np.loadtxt("reg_p2l.txt")

q_h = np.ndarray((Q_selected[0].shape[0], Q_selected[0].shape[1]+1))
q_h[:, :3] = Q_selected[0]
q_h[:, 3] = 1.0
q_h = q_h.T

Tq = np.linalg.inv(T).dot(q_h).T[:, :3]
print(Tq)
print(P_selected[0])

[[2.16015625 1.72265625 1.86595058]]
[[2.16015625 1.72265625 1.86595058]]
